In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#from keras.models import Sequential
#from keras.layers import Dense
import os

In [2]:
companies = [d for d in os.listdir('google_trends/')]

In [3]:
companies

['Tesla']

In [4]:
trends_files = {}
for each in companies:
    trends_files[each] = [f for f in os.listdir('google_trends/'+ each)]

In [5]:
trends_files

{'Tesla': ['Batterytechnology.csv',
  'Chargingstations.csv',
  'ElectricCars.csv',
  'ElectricPowertrain.csv',
  'ElectricSedans.csv',
  'ElectricSUVs.csv',
  'ElonMusk.csv',
  'Lithium-ionBatteries.csv',
  'Model3.csv',
  'ModelS.csv',
  'ModelX.csv',
  'Powerwall.csv',
  'SolarCity.csv',
  'SolarPanels.csv',
  'Solarpower.csv',
  'Tesla.csv',
  'TeslaRoadster.csv']}

In [6]:
tesla_trends = pd.DataFrame()
for company, trends_list in trends_files.items():
    for trend in trends_list:
        col_name = trend.split('.')[0]
        raw_data = pd.read_csv('google_trends/Tesla/' + trend)
        tesla_trends[col_name] = raw_data.drop(['Unnamed: 0', 'date'], axis=1)

In [7]:
# inputs
trends_x = tesla_trends

# label
trend_y = 'data/TSLA.csv'

In [7]:
orig = tesla_trends['Tesla'][:10]

In [17]:
tesla_differences = pd.DataFrame()

In [16]:
tesla_differences

,0
0,10
1,1


In [24]:
for i in range(len(orig)-1):
    x = orig[i]+orig[i+1]/2
    print(x)
    tesla_differences[0, i] = x

58.5
100.0
82.5
79.5
46.5
52.5
64.5
79.5
57.5


In [19]:
orig

0    21
1    75
2    50
3    65
4    29
5    35
6    35
7    59
8    41
9    33
Name: Tesla, dtype: int64

In [25]:
tesla_differences

,0,1,2,3,4,5,6,7,8,"(0, 0)",...,"(7, 0)","(8, 0)","(0, 1)","(0, 2)","(0, 3)","(0, 4)","(0, 5)","(0, 6)","(0, 7)","(0, 8)"


In [22]:
x = []
for each in trends_x:
    x.append(pd.read_csv(each))

In [32]:
data = np.array(x[0].merge(x[1], on='Date').merge(x[2], on='Date'))
# date, google, battery, solar, tesla

label = np.array(pd.read_csv(trend_y))

In [57]:
train_x = data[:330,:]
train_y = label[:330,:]
test_x = data[330:,:]
test_y = label[330:,:]

In [51]:
model = Sequential()
model.add(Dense(8, input_shape=(3,), activation='relu'))
model.add(Dense(8, input_shape=(3,), activation='relu'))
model.add(Dense(8, input_shape=(3,), activation='relu'))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [52]:
model.fit(train_x[:,1:], train_y[:,1:], epochs=20, verbose=1)

Epoch 1/20
330/330 [==============================] - 0s - loss: 197.3607 - acc: 0.0000e+00     
Epoch 2/20
330/330 [==============================] - 0s - loss: 114.9133 - acc: 0.0030         
Epoch 3/20
330/330 [==============================] - 0s - loss: 80.4653 - acc: 0.0000e+00     
Epoch 4/20
330/330 [==============================] - 0s - loss: 71.0179 - acc: 0.0030          
Epoch 5/20
330/330 [==============================] - 0s - loss: 67.5674 - acc: 0.0061          
Epoch 6/20
330/330 [==============================] - 0s - loss: 63.7213 - acc: 0.0061      

In [53]:
trainScore = model.evaluate(train_x[:,1:], train_y[:,1:], verbose=1)

 32/330 [=>............................] - ETA: 0s

In [59]:
testScore = model.evaluate(test_x[:,1:], test_y[:,1:], verbose=1)

 32/140 [=====>........................] - ETA: 0s

In [68]:
look_back = 10 # number of previous time steps to use as input to predict the next time period
trainPredict = model.predict(train_x[:,1:])
testPredict = model.predict(test_x[:,1:])

# shift train predictions for plotting
trainPredictPlot = np.empty_like(data[:,1:])
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
 
# shift test predictions for plotting
testPredictPlot = np.empty_like(data[:,1:])
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(data[:,1:])-1, :] = testPredict

ValueError: could not broadcast input array from shape (140,1) into shape (118,3)